In [3]:
import sys
file_path = '/home/samir/Desktop/rudraAnalytics/ENGINES/customer_analytics_engine'
sys.path.append(file_path)

import pandas as pd
from utils.date_util import convert_to_bs
from datetime import datetime
from utils.fuzzy_util import fuzz_check
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import joblib

def convert(x):
    try:
        return convert_to_bs(x)
    except:
        return None

def calculate_age(birthdate):
    try:
        return current_date.year - birthdate.year - ((current_date.month, current_date.day) < (birthdate.month, birthdate.day))
    except:
        return None
    
def clean_df(df):
    df = df[['dob', 
            'gender', 
            'last_qualification', 
            #    'father_occupation', 
            #    'division', 
            'completed_date']].copy()

    df['dob'] = df['dob'].apply(convert)
    df['Age'] = df['dob'].apply(calculate_age)
    df = df.dropna(subset=['Age'])
    df = df[df['gender'] != 'other']
    return df

def prepare_df(df):
    df['completed'] = ~df['completed_date'].isna()
    df['completed'] = df['completed'].astype(int)
    df = df.drop(['completed_date', 'dob'], axis=1)

    enum_set = ['Inter', '+2', 'NEB', 'BBA', 'BBS', 'Isc', 'BA', 'BSc', 'Civil', 'SEE', 'Undergraduate']
    def fuzz_check_udf(x):
        return fuzz_check(x, enum_set, 55, 'unknown_profession')

    df['last_qualification'] = df['last_qualification'].fillna('unknown_profession').apply(fuzz_check_udf)
    gender_encoded = pd.get_dummies(df['gender'], drop_first=True)
    df = pd.concat([df, gender_encoded], axis=1)

    label_encoder = LabelEncoder()
    df['lq'] = label_encoder.fit_transform(df['last_qualification'])

    return df

def finalize(df):
    df = df[['male','Age','lq', 'completed']].to_numpy()
    scaler = StandardScaler()
    return scaler.fit_transform(df)

k= 5
knn = KNeighborsClassifier(n_neighbors=k)
base = pd.read_csv("/home/samir/Desktop/rudraAnalytics/sub_projects/NEEM/data/factored/students.csv")

current_date = datetime.now()

df = clean_df(base)
df = prepare_df(df)
# df = finalize(df)
# df = pd.DataFrame(df)
df = df[['male','Age','lq', 'completed']]

In [4]:
sn

,male,Age,lq,completed
0,0,26.0,6,1
1,0,21.0,6,1
2,0,24.0,11,0
3,1,27.0,11,1
4,0,17.0,0,0
...,...,...,...,...
1069,1,13.0,11,1
1070,1,16.0,11,1
1071,0,26.0,11,1
1072,0,23.0,11,1
